# MARATONA BEHIND THE CODE 2020

## DESAFIO 6: LIT

## Testes com modelo H2O
- Autor: Rodrigo Lima
- LinkedIN: https://www.linkedin.com/in/rodrigolima82/


![title](virtual_assistance.png)

# Parte 01. Importando as bibliotecas

In [1]:
# Instalando o pacote h2o
!pip install h2o

In [27]:
# Importando as bibliotecas
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

# Inicializando o cluster
h2o.init()
h2o.cluster().show_status()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,13 days 23 hours 43 mins
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,"21 days, 7 hours and 55 minutes"
H2O_cluster_name:,H2O_from_python_unimed_arnacy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,14.46 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


H2O_cluster_uptime:,13 days 23 hours 43 mins
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,"21 days, 7 hours and 55 minutes"
H2O_cluster_name:,H2O_from_python_unimed_arnacy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,14.46 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


# Parte 02. Carregando o Dataset

In [32]:
# Carregando o dataset da competição
dados = pd.read_csv('../data/training_dataset.csv')

# Remove as variáveis de ID e unique
dados.drop(['id','importante_ter_certificado'], axis = 1, inplace = True)

# Visualizando os primeiros registros
print(dados.shape)
dados.head()

(15857, 13)


,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
0,Especialização,UNIP,Analista Senior,Eletrodomesticos,1.0,1.0,6.0,Google,28.0,5.0,4.0,0.0,perfil6
1,MBA,NaN,Supervisor,e-commerce,0.0,0.0,6.0,NaN,66.0,36.0,34.0,0.0,perfil2
2,Bacharelado,UNICAMP,Outros,e-commerce,1.0,1.0,10.0,Saint Paul,27.0,27.0,27.0,0.0,perfil2
3,SEM FORMAÇÃO,UFRN,Outros,Eletroeletronicos,NaN,1.0,5.0,Instagram,29.0,10.0,6.0,0.0,perfil5
4,Tecnólogo,INSPER INSTITUTO DE ENSINO E PESQUISA,SEM EXPERIÊNCIA,Eletrodomesticos,NaN,0.0,7.0,Google,67.0,49.0,44.0,0.0,perfil2


In [33]:
# Importando os datasets   
train = h2o.H2OFrame(dados)

#train = h2o.import_file('../data/training_dataset.csv')
test = h2o.import_file('../data/to_be_scored.csv')

# Convertendo a coluna target em fator
train['categoria'] = train['categoria'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Parte 03. Executando o H2OAutoML

In [4]:
# Indicando as colunas preditoras e a target
x = train.columns
y = 'categoria'

# Converter o target em fator
train[y] = train[y].asfactor()

# Executa o AutoML para 15 modelos (limitado a 1h de duracao máxima)
aml = H2OAutoML(max_models=15, seed=42)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |█████████████████
18:53:35.480: XRT_1_AutoML_20200830_185220 [DRF XRT (Extremely Randomized Trees)] failed: java.lang.AssertionError

███████████████████████████████████████| 100%


In [36]:
# Indicando as colunas preditoras e a target
x = train.columns
y = 'categoria'

# Converter o target em fator
train[y] = train[y].asfactor()

# Executa o AutoML para 15 modelos (limitado a 1h de duracao máxima)
aml = H2OAutoML(max_models=15, seed=42)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


# Parte 04. Avaliação do Desempenho dos Modelos

In [5]:
# AutoML Leaderboard
lb = aml.leaderboard

# Exibir as colunas do resultado dos modelos
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_grid__1_AutoML_20200830_185220_model_1,0.209177,0.78819,0.451545,0.203893
StackedEnsemble_BestOfFamily_AutoML_20200830_185220,0.21009,0.751573,0.451764,0.204091
StackedEnsemble_AllModels_AutoML_20200830_185220,0.210296,0.747045,0.449204,0.201784
XGBoost_2_AutoML_20200830_185220,0.210657,0.798299,0.454273,0.206364
XGBoost_grid__1_AutoML_20200830_185220_model_2,0.210875,0.757183,0.45,0.2025
GBM_5_AutoML_20200830_185220,0.211489,0.759851,0.451128,0.203516
GBM_1_AutoML_20200830_185220,0.21149,0.761393,0.452897,0.205116
XGBoost_3_AutoML_20200830_185220,0.211558,0.765774,0.451151,0.203538
GBM_2_AutoML_20200830_185220,0.211969,0.763162,0.451385,0.203749
XGBoost_1_AutoML_20200830_185220,0.212085,0.804907,0.453752,0.205891


In [6]:
# Detalhes do melhor modelo
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20200830_185220_model_1


Model Summary: 


,,number_of_trees
0,,30.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.14331103171453027
RMSE: 0.37856443535352113
LogLoss: 0.4642815764389068
Mean Per-Class Error: 0.1846752254140663

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,perfil1,perfil2,perfil3,perfil4,perfil5,perfil6,Error,Rate
0,1864.0,298.0,37.0,65.0,55.0,153.0,0.245955,608 / 2.472
1,128.0,3624.0,60.0,89.0,93.0,221.0,0.140214,591 / 4.215
2,28.0,80.0,827.0,22.0,28.0,68.0,0.214625,226 / 1.053
3,44.0,116.0,17.0,1342.0,40.0,102.0,0.192053,319 / 1.661
4,42.0,112.0,23.0,46.0,1567.0,127.0,0.182577,350 / 1.917
5,134.0,237.0,57.0,92.0,82.0,3937.0,0.132628,602 / 4.539
6,2240.0,4467.0,1021.0,1656.0,1865.0,4608.0,0.170020,2.696 / 15.857



Top-6 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
0,,2020-08-30 18:53:43,7.733 sec,0.0,0.833333,1.791759,0.713754
1,,2020-08-30 18:53:43,8.114 sec,5.0,0.501796,0.774016,0.186164
2,,2020-08-30 18:53:43,8.419 sec,10.0,0.435454,0.635017,0.184461
3,,2020-08-30 18:53:44,8.750 sec,15.0,0.414797,0.576185,0.181623
4,,2020-08-30 18:53:44,9.167 sec,20.0,0.401344,0.533018,0.178281
5,,2020-08-30 18:53:45,9.584 sec,25.0,0.390035,0.497338,0.173929
6,,2020-08-30 18:53:45,9.922 sec,30.0,0.378564,0.464282,0.170020



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,modulos_finalizados,37300.230469,1.000000,0.523098
1,certificados,12944.977539,0.347048,0.181540
2,modulos_iniciados,8972.091797,0.240537,0.125825
3,id,3881.627197,0.104064,0.054436
4,total_modulos,3219.752930,0.086320,0.045154
5,horas_semanais_estudo,1619.382812,0.043415,0.022710
6,pretende_fazer_cursos_lit,304.226532,0.008156,0.004266
7,interesse_mba_lit,264.560120,0.007093,0.003710
8,como_conheceu_lit.Outros,114.406631,0.003067,0.001604
9,graduacao.MBA,97.030525,0.002601,0.001361



See the whole table with table.as_data_frame()


In [37]:
# AutoML Leaderboard
lb = aml.leaderboard

# Exibir as colunas do resultado dos modelos
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_grid__1_AutoML_20200831_223055_model_1,0.20933,0.78299,0.450745,0.203171
StackedEnsemble_AllModels_AutoML_20200831_223055,0.209456,0.751433,0.451442,0.2038
StackedEnsemble_BestOfFamily_AutoML_20200831_223055,0.210134,0.75651,0.45394,0.206062
XGBoost_grid__1_AutoML_20200831_223055_model_2,0.210363,0.756217,0.449568,0.202112
XGBoost_3_AutoML_20200831_223055,0.210576,0.763269,0.449999,0.202499
GBM_3_AutoML_20200831_223055,0.210588,0.768875,0.451162,0.203547
GBM_1_AutoML_20200831_223055,0.211202,0.760078,0.451656,0.203994
XGBoost_2_AutoML_20200831_223055,0.211264,0.78914,0.452461,0.204721
GBM_4_AutoML_20200831_223055,0.211486,0.785794,0.444726,0.197781
GBM_2_AutoML_20200831_223055,0.211637,0.762477,0.455028,0.20705


In [38]:
# Detalhes do melhor modelo
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20200831_223055_model_1


Model Summary: 


,,number_of_trees
0,,30.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.15393316092185275
RMSE: 0.39234316729344576
LogLoss: 0.5067029943779562
Mean Per-Class Error: 0.18927013744768253

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,perfil1,perfil2,perfil3,perfil4,perfil5,perfil6,Error,Rate
0,1843.0,312.0,37.0,68.0,55.0,157.0,0.254450,629 / 2.472
1,134.0,3603.0,64.0,95.0,96.0,223.0,0.145196,612 / 4.215
2,28.0,80.0,828.0,24.0,28.0,65.0,0.213675,225 / 1.053
3,44.0,116.0,24.0,1334.0,40.0,103.0,0.196869,327 / 1.661
4,40.0,112.0,24.0,54.0,1555.0,132.0,0.188837,362 / 1.917
5,138.0,242.0,57.0,96.0,87.0,3919.0,0.136594,620 / 4.539
6,2227.0,4465.0,1034.0,1671.0,1861.0,4599.0,0.175002,2.775 / 15.857



Top-6 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
0,,2020-08-31 22:32:35,8.872 sec,0.0,0.833333,1.791759,0.713754
1,,2020-08-31 22:32:35,9.195 sec,5.0,0.504419,0.784522,0.187614
2,,2020-08-31 22:32:35,9.486 sec,10.0,0.439435,0.652600,0.185659
3,,2020-08-31 22:32:36,9.734 sec,15.0,0.420805,0.600974,0.183200
4,,2020-08-31 22:32:36,9.977 sec,20.0,0.409598,0.563571,0.180236
5,,2020-08-31 22:32:36,10.359 sec,25.0,0.400470,0.532513,0.177839
6,,2020-08-31 22:32:37,10.711 sec,30.0,0.392343,0.506703,0.175002



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,modulos_finalizados,38575.234375,1.000000,0.559833
1,certificados,12520.117188,0.324564,0.181701
2,modulos_iniciados,8417.528320,0.218211,0.122162
3,total_modulos,3833.623047,0.099380,0.055636
4,horas_semanais_estudo,1786.471924,0.046311,0.025927
5,pretende_fazer_cursos_lit,317.497009,0.008231,0.004608
6,interesse_mba_lit,291.129425,0.007547,0.004225
7,como_conheceu_lit.Outros,114.306175,0.002963,0.001659
8,graduacao.MBA,106.753082,0.002767,0.001549
9,graduacao.Bacharelado,106.266426,0.002755,0.001542



See the whole table with table.as_data_frame()


# Parte 05. Realizar previsões com dados de teste

In [12]:
# Gerar as previsoes no dataset de teste
preds = aml.leader.predict(test)

# Salvar o resultado no dataset de teste
result = preds[:,0]
test['target'] = result

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [39]:
# Gerar as previsoes no dataset de teste
preds = aml.leader.predict(test)

# Salvar o resultado no dataset de teste
result = preds[:,0]
test['target'] = result

xgboost prediction progress: |████████████████████████████████████████████| 100%


# Parte 6. Submissão do Desafio 6

In [14]:
# Gravar o arquivo de submissão ao Desafio
h2o.export_file(test,'results.csv')

Export File progress: |███████████████████████████████████████████████████| 100%


In [41]:
# Gravar o arquivo de submissão ao Desafio
h2o.export_file(test,'results_v3.csv')

Export File progress: |███████████████████████████████████████████████████| 100%
